Extracting User's Google Calendar

In [1]:
import methods
import formats
import json

google_calendar = methods.get_google_calendar_service()
calendar = methods.extractCalendar(google_calendar)
client = methods.set_up_ChatGPT(calendar)

Based on the calendar information provided, there is one event scheduled for today:

1. Event: 545 Project Meeting
   Start Time: 2024-03-25T16:00:00-04:00
   End Time: 2024-03-25T18:00:00-04:00

In [11]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Get User's Query

In [4]:
query = input()
prompt2 = "Extract intent from the following query meant for a virtual calendar assistant - " + query + ". Your response should be a single word that describes the intent of the query, and should be one of the following: createEvent, moveEvent, deleteEvent, checkSchedule, listEvents, updateEvent."
stream = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": prompt2}],
    stream=True,
)
intent = ''
for chunk in stream:
    intent += chunk.choices[0].delta.content or ""
print(intent)

createEvent


In [18]:
if intent == 'moveEvent':
    response = client.chat.completions.create(
        model = 'gpt-3.5-turbo',
        messages = [{'role': 'user', 'content': query}],
        functions = formats.moveEvent_format,
    )


    json_response = json.loads(response.choices[0].message.function_call.arguments)


    entities = dict(json_response)
    if not entities.get('new_time', ''):
        prompt = 'Here is my calendar for the week: ' + calendar + '\n' + ' Use it to find a new time to schedule the following event, taking into account the type of event (name)' + str(json_response) + 'Your response should only output new_time as a single string where new_time represents The time to which the event is to be moved. Use 12HR format, example output: 10:00 AM.'
        response2 = client.chat.completions.create(
            model = 'gpt-3.5-turbo',
            messages = [{'role': 'user', 'content': prompt}],
        )
        output = response2.choices[0].message.content
        print(output)
        entities['new_time'] = output
        methods.moveEvent(entities)
    else:
        methods.moveEvent(entities)






In [6]:
if intent == 'createEvent':
    response = client.chat.completions.create(
        model = 'gpt-3.5-turbo',
        messages = [{'role': 'user', 'content': query + "based on " + calendar}],
        functions = formats.createEvent_format,
    )

    json_response = json.loads(response.choices[0].message.function_call.arguments)
    print(json_response)
    entities = dict(json_response)


    if not entities.get('time', ''):
        prompt = 'Here is my calendar for the week: ' + calendar + '\n' + ' Use it to find a time to schedule the following event, taking into account the type of event (name)' + str(json_response) + 'Your response should only output new_time as a single string where new_time represents The time to which the event is to be moved. Use 12HR format, example output: 10:00 AM.'
        response2 = client.chat.completions.create(
            model = 'gpt-3.5-turbo',
            messages = [{'role': 'user', 'content': prompt}],
        )
        output = response2.choices[0].message.content
        print(output)
        entities['time'] = output
        methods.createEvent(entities)
    else:
        methods.createEvent(entities)


{'description': 'Dinner with friends', 'date': '2024-03-26', 'time': '6:00 PM', 'duration': '3 hours'}
2024-03-26 18:00:00
Event created: Dinner with friends on 2024-03-26 at 6:00 PM for 3 hours hours.


In [ ]:
#TODO: Add deleteEvent, listEvents, updateEvent, RecurringEvents intents

In [ ]:
if intent == "checkSchedule":
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": query}],
        functions= formats.checkSchedule_format,
    )
    json_response = json.loads(response.choices[0].message.function_call.arguments)
    entities = dict(json_response)
    st.write("Entities: ", entities)
    methods.checkSchedule(entities)
